In [37]:
from keras.models import Model
from keras.layers import *
import os
import tensorflow as tf

#A method to convert keras to tensorflow
def keras_to_tf(keras_model, output_dir, model_name, out_prefix = "output_", log_tensorboard=True):
    if os.path.exists(output_dir) == False:
        os.mkdir(output_dir)

    out_nodes = []

    for i in range(len(keras_model.outputs)):
        out_nodes.append(out_prefix + str(i + 1))
        tf.identity(keras_model.output[i], out_prefix + str(i + 1))

    sess = K.get_session()
    
    from tensorflow.python.framework import graph_util, graph_io

    init_graph = sess.graph.as_graph_def()

    main_graph = graph_util.convert_variables_to_constants(sess, init_graph, out_nodes)

    graph_io.write_graph(main_graph, output_dir, name=model_name, as_text=False)

    if log_tensorboard:
        from tensorflow.python.tools import import_pb_to_tensorboard

        import_pb_to_tensorboard.import_to_tensorboard(
            os.path.join(output_dir, model_name), output_dir)
        
    
    """
    We explicitly redefine the Squeezent architecture since Keras has no predefined Squeezenet
    """
def squeezenet_fire_module(inputs, input_channel_small=16, input_channel_large=64):

    channel_axis = 3

    inputs = Conv2D(input_channel_small, (1,1), padding="valid" )(inputs)
    inputs = Activation("relu")(inputs)

    input_branch_1 = Conv2D(input_channel_large, (1,1), padding="valid" )(inputs)
    input_branch_1 = Activation("relu")(input_branch_1)

    input_branch_2 = Conv2D(input_channel_large, (3, 3), padding="same")(inputs)
    input_branch_2 = Activation("relu")(input_branch_2)

    inputs = concatenate([input_branch_1, input_branch_2], axis=channel_axis)

    return inputs




def SqueezeNet(input_shape=(128,128,3)):



    image_input = Input(shape=input_shape)


    network = Conv2D(64, (3,3), strides=(2,2), padding="valid")(image_input)
    network = Activation("relu")(network)
    network = MaxPool2D( pool_size=(3,3) , strides=(2,2))(network)

    network = squeezenet_fire_module(inputs=network, input_channel_small=16, input_channel_large=32)
    network = squeezenet_fire_module(inputs=network, input_channel_small=16, input_channel_large=32)
    network = MaxPool2D(pool_size=(3,3), strides=(2,2))(network)

    #network = squeezenet_fire_module(inputs=network, input_channel_small=32, input_channel_large=128)
    #network = squeezenet_fire_module(inputs=network, input_channel_small=32, input_channel_large=128)
    network = MaxPool2D(pool_size=(3, 3), strides=(2, 2))(network)

    #network = squeezenet_fire_module(inputs=network, input_channel_small=48, input_channel_large=192)
    #network = squeezenet_fire_module(inputs=network, input_channel_small=48, input_channel_large=192)
    #network = squeezenet_fire_module(inputs=network, input_channel_small=64, input_channel_large=256)
    #network = squeezenet_fire_module(inputs=network, input_channel_small=64, input_channel_large=256)

    #Remove layers like Dropout and BatchNormalization, they are only needed in training
    #network = Dropout(0.5)(network)

    network = Conv2D(1000, kernel_size=(1,1), padding="valid", name="last_conv")(network)
    network = Activation("relu")(network)

    network = GlobalAvgPool2D()(network)
    #network = Activation("softmax",name="output")(network)
    
    network = Flatten()(network)
    network = Dense(units = 1, activation='sotfmax', )


    input_image = image_input
    model = Model(inputs=input_image, outputs=network)

    return model

In [38]:
keras_model = SqueezeNet()

In [39]:
keras_model.load_weights("model1.h5")

output_dir = os.path.join(os.getcwd(),"checkpoint")
keras_to_tf(keras_model, output_dir=output_dir, model_name="squeezenet.pb")

print("Model Saved!!!")

ValueError: You are trying to load a weight file containing 4 layers into a model with 8 layers.

In [20]:
from datetime import datetime
date = datetime.now()

datenow = date.ctime()
datenow[4:]

'Jun  9 16:09:49 2020'